In [1]:
import glob,os
import pandas as pd
import numpy as np
import pandas as pd
import os,glob
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append(os.path.expanduser("~") + "/collproc/")
from raw import *
from coll import *
sys.path.append(os.path.expanduser("~"))
from ss2csv.ss2csv import file2table,cleancols
import matplotlib.pyplot as plt

In [2]:
DIR = os.path.expanduser("~") + "/Dropbox/lml/genome_measurements/"

In [3]:
allcolor = dirwalk(DIR,require='lor',omit=['Tests','flags','Issues'])

In [4]:
df = pd.DataFrame()

for f in allcolor:
    tmp = cleancols(file2table(f))
    tmp['fname'] = os.path.basename(f)
    df = pd.concat([df,tmp])

In [5]:
df = df.reset_index(drop=True)

In [6]:
# occasionally, there is something in these columns, but not here

del df['SAMPLE_ID2']
del df['SAMPLE_ID3']

# LUV conversion

For some reason, we occasionally have LUV measurements instead of LAB. But we can convert easily:

In [7]:
from colormath.color_objects import LabColor,LuvColor
from colormath.color_conversions import convert_color

In [8]:
def luv2lab(i):
    l = df.LUV_L.loc[i]
    u = df.LUV_U.loc[i]
    v = df.LUV_V.loc[i]
    luv = LuvColor(l,u,v,observer='2',illuminant='d65')
    lab = convert_color(luv,LabColor)
    return lab.get_value_tuple()

In [9]:
luvidxs = df.index[df.LUV_L.notnull()]

In [10]:
for luvidx in luvidxs:
    lab = luv2lab(luvidx)
    df.LAB_L.loc[luvidx] = lab[0]
    df.LAB_A.loc[luvidx] = lab[1]
    df.LAB_B.loc[luvidx] = lab[2]

In [11]:
del df['LUV_L']
del df['LUV_U']
del df['LUV_V']

# Parse Sample ID

In [12]:
df = pd.concat([df,pd.DataFrame(list(df.SAMPLE_ID1.apply(parse_sample_id)))],axis=1)

In [13]:
df = df.drop_duplicates()

In [14]:
df = df.reset_index(drop=True)

In [15]:
pkl(df,'raw.pkl')